In [1]:
# Loading libraries for the course
import os
import random
import urllib.request                       # Library to download data
import numpy as np                          # Library for array manipulation
import pandas as pd                         # Library to manipulate data frames
import tifffile                             # Library to store numpy arrays as TIFF
import pathlib                              # Library to work with file paths
from pathlib import Path                    # Library to work with file paths
import matplotlib.pyplot as plt             # Library used for plotting
from matplotlib.patches import Rectangle    # Module to plot a rectangle in the image
from matplotlib import animation            # Module to plot animations
import seaborn as sn                        # Library for advanced plotting
import skimage                              # Library for image manipulation (scikit-image)
from skimage.io import imread, imshow       # Module from skimage
from skimage import io, measure, filters, morphology, segmentation, color, feature
from skimage.filters import gaussian, threshold_otsu, difference_of_gaussians
from skimage.morphology import (binary_dilation, binary_erosion, binary_closing, 
                                binary_opening, disk, remove_small_objects)
from skimage.draw import polygon
from skimage.measure import regionprops
from skimage.color import label2rgb
from skimage.segmentation import watershed
from scipy import ndimage as ndi              # Distance Transform
from skimage.feature import peak_local_max    # Local maxima in a matrix
from PIL import Image
import cellpose

%matplotlib inline

from ipywidgets import interactive, HBox, VBox, Layout
import ipywidgets as widgets




In [ ]:
#Metadata we want at the end of the analysis

metadata = {'frame_number':[], 'cell_number':[], 'number_of_viable_cells':[], 'cell_size':[], 'nucleus_size':[], 'protein_concentration':[], 'number_of_mRNA_in_nucleus':[], 'number_of_mRNA_in_cytoplasm':[], 'number_of_transcription_sites':[], 'intensity_of_transcription_sites':[]}


In [ ]:
#i want to open a folder of the tiff files and open the mask files too of the nuclei that Wictoria did

import os 
path = 'Drug1_1000nM/' #this is the path to the folder where the tiff files are
files = os.listdir(path) #this will list all the files in the folder 
for file in files: #this will loop over all the files in the folder
    open_tiff = imread(path + file) #this will open the tiff file
    img = open_tiff

In [ ]:
#NUMBER OF TRANSCRIPTION SITES

#selecting the blue channel
img_B = img[1, :, :, 2]

#show the original image with the blue channel showing the transcription sites
plt.imshow(img_B)
plt.title('Original image')
plt.show()

#apply gaussian filter and show the image  
img_B_blur = gaussian(img_B, sigma=5)
plt.imshow(img_B_blur)
plt.title('Blurred image')
plt.show()

#apply difference of gaussians filter and show the image 
img_B_dog = difference_of_gaussians(img_B_blur, low_sigma=5, high_sigma=10)
plt.imshow(img_B_dog)
plt.title('Difference of Gaussians')
plt.show()

#threshold the image to see only the transcription sites
threshold_B = threshold_otsu(img_B_dog)
img_B_bin = img_B_dog > threshold_B
plt.imshow(img_B_bin)
plt.title('Binarized image')
plt.show()

#the transcription sites must be inside the nucleus, so I will use the nucleus mask to remove the transcription sites that are outside the nucleus
#i need to save the nucleus mask as a tiff file in Wictoria's code and then load it here to use it

#EXAMPLE
#load the nucleus mask
nucleus_mask = imread('nucleus_mask.tif')
plt.imshow(nucleus_mask)
plt.title('Nucleus mask')
plt.show()

#apply the nucleus mask to the binarized image
img_B_bin_nucleus = img_B_bin * nucleus_mask
plt.imshow(img_B_bin_nucleus)
plt.title('Binarized image with nucleus mask')
plt.show()

#label the transcription sites
img_B_label = measure.label(img_B_bin_nucleus)
plt.imshow(img_B_label)
plt.title('Labeled transcription sites')
plt.show()

#count the number of transcription sites INSIDE the nucleus
num_transcription_sites = img_B_label.max()
print('Number of transcription sites:', num_transcription_sites)

#measure the intensity of the transcription sites
props_B = measure.regionprops(img_B_label)

#collect the intensity of the transcription sites
transcription_sites_intensity = [region.mean_intensity for region in props_B]

#add the number of transcription sites and the intensity of the transcription sites to the metadata
metadata['number_of_transcription_sites'] = num_transcription_sites



